In [3]:
from langchain_openai import ChatOpenAI
from getpass import getpass
import pandas as pd
from tqdm import tqdm

#course_api_key= "Введите ваш API ключ с курса"
course_api_key = getpass(prompt='Введите API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(api_key=course_api_key, model='gpt-4o-mini',
                 base_url="https://aleron-llm.neuraldeep.tech/")

In [4]:
df = pd.read_csv('submission100lines.csv',encoding='utf-8')
df.head()

,amount,text_id,text
0,2,14205200,Сниму жилье. 500-600 рублей сутки. Эконом клас...
1,6,319097075,ищем жилье в центре недалеко от моря с 23.07-0...
2,4,98881311,Ищем жилье на период с 18-28июля..на две семьи...
3,4,44587027,2 семьи по 3 человека (2 взрослых и ребенок) с...
4,3,352802829,Сниму недорогое жилье в лазаревском на 6 чел 3...


In [ ]:
prompt_template = """Ты - профессиональный риелтор с 10-ти летним опытом работы. Твоя задача - понять из текста сколько человек
будут жить в квартире. Количество человек может быть написано как текстом - один человек,три человека, так и числом - 2 человека.
В некоторых примерах необходимо посчитать количество человек - Сниму на 2 семьи по 3 человека (ответ будет 2 * 3 = 6 человек).
Каждый взрослый или ребенок считается за одного человека.
Формат ответа:
- целое число (1,2,3 и тд).
Пример: Две семьи, молодая пара c ребенком и их родители снимут квартиру на 5 дней
Ход рассуждений:
1. Молодая пара - это два человека
2. Ребенок - это один человек
3. Родители пары - 2 человека
4. 2 + 1 + 2 = 5 человек

Ответ: 5

Context: {text_input}

###

Question: Сколько человек будет проживать в квартире?

###

Answer: ...
"""

In [11]:
amount_list = [] # Список, где будем хранить ответы модели

for text_input in tqdm(df['text']):
    prompt = prompt_template.format(text_input=text_input) # Добавляем сообщение в промпт
    amount = llm.invoke(prompt).content # Ответ модели
    amount_list.append(amount) # Добавляем ответ в список

# перед записью ответа в датафрейм, проверьте какого типа ответы
type(amount_list[0])

100%|██████████| 100/100 [02:15<00:00,  1.36s/it]


str

In [21]:
amount_list
answers = map(lambda x: x.split('\n\n')[-1].split(': ')[-1],amount_list)
cleaned_answers = map(lambda x: x.replace('.',''),answers)
int_answers = map(int,cleaned_answers)
df['amount'] = list(int_answers)
df.to_csv('2_1_8_solution.csv', index=False)